In [ ]:
import folium
import data.DownloadData as DD
import data.DataSorting as DS
import json
import geojson
from urllib.request import urlopen
from geojson_utils import centroid

In [ ]:
with urlopen('https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/cantons.geojson') as f:
    stations = geojson.load(f)

In [ ]:
# récupération des données sur les stations-service
DD.get_ten_Mins_Data()
DD.get_dailyData()

# on récupère le J-5 pour observer la variation 
chaine = DD.get_chaine()

# calcul des prix
st_prices = DS.DictCantons("PrixCarburants_instantane.xml")
st_prices_B = DS.DictCantons("PrixCarburants_quotidien_" + chaine + ".xml")

# calcul de la variation
variations = {key: st_prices[key] - st_prices_B.get(key, 0) for key in st_prices.keys()}

In [ ]:
s = folium.Map(location=[46.232192999999995, 2.209666999999996], zoom_start=5)

for i in range (0, len(stations["features"])):
    if stations["features"][i].geometry.type == 'Polygon':
        box_str = str(stations["features"][i].geometry)
        box = json.loads(box_str)
        center = centroid(box)
        loc = [center['coordinates'][1], center['coordinates'][0]]
        nom_v = stations["features"][i]["properties"]["nom"]
        if nom_v in variations:
            if variations[nom_v] < 0:
                folium.Marker(location = loc,
                                  popup = f'{nom_v}</br>{round(variations[nom_v], 4)}',
                                  icon = folium.features.CustomIcon('down.png',icon_size=(12,12))).add_to(s)
            elif 0 < variations[nom_v] <= max(variations.values())/12:
                folium.Marker(location = loc,
                                  popup = f'{nom_v}</br>+{round(variations[nom_v], 4)}',
                                  icon = folium.features.CustomIcon('up_1.png',icon_size=(12,12))).add_to(s)
            elif max(variations.values())/12 < variations[nom_v] <= max(variations.values()):
                folium.Marker(location = loc,
                                  popup = f'{nom_v}</br>+{round(variations[nom_v], 4)}',
                                  icon = folium.features.CustomIcon('up_2.png',icon_size=(12,12))).add_to(s)
            else:
                folium.Marker(location = loc,
                                  popup = nom_v,
                                  icon = folium.features.CustomIcon('equal.png',icon_size=(16,16))).add_to(s)

s